In [1]:
#libaraies
import copy
import numpy as np
import math

# clusters
k = 2

# memory
m = 6

# dimensions of data
d = 2

# initial time
time = 1 

# max duration of running stream
maxtime = 60

# dict containing centroids at each level 
level = {}

#randomnly creating an array of x and y corrodinate of d dimension
def online_data(): 
    
    x = np.random.randint(10,70,(1,d))  
    # convert to np.float32 
    x = np.float32(x) 
    return x[0]

# finds a single centroid with given centroids and thier frequencies in d dimension
def weighted_centroid(weighted_arr):
    
    sum = num_points = 0
    median = [0]*d
    for pair in weighted_arr:
        num_points += pair[1]
    
    for i in range(d):
        weighted_arr.sort(key=lambda x: x[0][i])
        sum = flag1 = flag2 = 0
        for pair in weighted_arr:
            if(flag1 == 0 and math.floor((num_points+1)/2) >= sum and math.floor((num_points+1)/2) <= sum+pair[1]):
                median[i] += pair[0][i]
                flag1 = 1
            if(flag2 ==0 and num_points%2==0 and (math.floor(num_points/2)+1) >= sum and (math.floor(num_points/2)+1) <= sum+pair[1]):
                median[i] += pair[0][i]
                flag2 = 1
            sum += pair[1]

        if(num_points%2==0):
            median[i] /= 2
            
    return [median,num_points]
      
# finds k clusters among the clusters in the range query
def weighted_kmedian(k,weighted_list):
    
    itr = 100
    cluster_list = []
    for i in range(k):
        temp_list = copy.deepcopy(weighted_list[i])
        cluster_list.append(temp_list)
    
    while(itr > 0):
        
        itr -= 1
        for i in range(k):
            cluster_list[i][1] = 0
            
        weighted_arr = [[] for i in range(k)]
        
        for i in range(len(weighted_list)):
            mindist = math.inf
            ind = 0
            for j in range(k):
                distance = dist_v(weighted_list[i][0],cluster_list[j][0])
                mindist = min(mindist,distance)
                if(mindist==distance):
                    ind = j
            weighted_arr[ind].append(weighted_list[i])
            
        for i in range(k):
            cluster_list[i] = weighted_centroid(weighted_arr[i])
                
    return cluster_list

# squaring of two points   
def dist(x,y):
    return (x-y)**2

# finds the euclidean distance of 2 vectors of d dimensions,
def dist_v(x,y):
    sum = 0
    for i in range(d):
        sum += dist(x[i],y[i])
    sum = math.sqrt(sum/d)
    return sum

# shows the result of the program
def show_result(cluster_list):
    
    print("Final K clusters:\n")
    for i in range(len(cluster_list)):
        print("Cluster "+str(i+1),"  Cluster Center(d dimension):  ", str(cluster_list[i][0]),"  Data points: ", str(cluster_list[i][1])) 

# stream algorithm
def stream():

    if(k > m):
        print("Memory should be more than clusters!")
        return

    if(m%k!=0):
        print("Memory should be divisible by clusters!")
        return 
    
    global time
    
    while(time <= maxtime):

        x = online_data()

        if(0 not in level):
            level[0] = []

        # initially, new data point is in level[0] 
        temp_list = [x,1]
        level[0].append(temp_list)

        index = 0

        while(1):

            if(len(level[index]) >= m):
                weighted_list = []
                for i in range(m):
                    weighted_list.append(level[index][i])
                for i in range(m):
                    level[index].pop(0)
                if(index+1 not in level):
                    level[index+1] = []
                    
                # passing m weighted d dimensional centers to find k clusters from them 
                cluster_list = weighted_kmedian(k,weighted_list)
                for pair in cluster_list:
                    level[index+1].append(pair) # adding new clusters in higher level
            else:
                break
                
            index += 1
            
        time += 1
           
    # adding the leftout/whole clusters to find the k final clusters
    weighted_list = []
    for lvl in level:
        for el in level[lvl]:
            weighted_list.append(el)
            
    clustered_list = weighted_kmedian(k,weighted_list)
    
    show_result(clustered_list)
    
    
stream()


Final K clusters:

Cluster 1   Cluster Center(d dimension):   [50.5, 39.25]   Data points:  21
Cluster 2   Cluster Center(d dimension):   [27.5, 47.25]   Data points:  39
